In [ ]:
# Predicting two outputs
# described by a probability distribution (a mean and a standard deviation)
# --> solution 1 : two final Dense layers
# https://stackoverflow.com/questions/71300786/tensorflow-probability-want-nn-to-output-multiple-distributions

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
tf.__version__

import tensorflow_probability as tfp

print(tf.__version__)

import pandas as pd
df = pd.DataFrame()
df['input'] = np.random.uniform(-10.0, 10.0, 5000)

# Simple rules:
#   If input is positive:
#     pos_points = input * input
#     neg_points = -0.5 * input
#
#   If input is negative:
#     pos_points = -0.5 * input
#     neg_points = -input * input

df['predict_1'] = df['input'].apply(lambda x: x*x if x > 0 else x * -0.5)
# df['predict_2'] = df['input'].apply(lambda x: x*x*-1 if x < 0 else -x * 0.5)

print(df.head())

target = pd.concat([df.pop(x) for x in ['predict_1']], axis=1)

tfd = tfp.distributions

print(len(df.columns))

# Build a simple model to go from input to the two outputs
def get_df_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, input_shape=[len(df.columns),], activation='relu'), # Should only be one input, so [1,]
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(2 * len(target.columns)), # there is 1 value we're predicting, we want a mean + standard deviation, so TWO outputs
    tfp.layers.DistributionLambda(
      lambda t: tfd.Normal(loc=t[..., :1],
                           scale=1e-3 + tf.math.softplus(0.05 * t[...,1:]))
    )
  ])

  negloglik = lambda y, rv_y: -rv_y.log_prob(y)

  model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01), loss=negloglik)
  return model

model = get_df_model()
model.summary()
model.fit(df, target, epochs=10)

2.8.0
      input  predict_1
0  3.498475  12.239330
1  2.134057   4.554200
2  5.456111  29.769145
3  0.263752   0.069565
4  7.399465  54.752085
1
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 10)                20        
                                                                 
 dense_16 (Dense)            (None, 10)                110       
                                                                 
 dense_17 (Dense)            (None, 2)                 22        
                                                                 
 distribution_lambda_5 (Dist  ((None, 1),              0         
 ributionLambda)              (None, 1))                         
                                                                 
Total params: 152
Trainable params: 152
Non-trainable params: 0
__________________________________________

In [ ]:
# Test results:
yhat = model(df.to_numpy(dtype=np.float32))

print(df.head(1))
print(target.head(1))

print("----- Predictions:")
print("mean", yhat.mean()[0])
print("stddev", yhat.stddev()[0])

In [ ]:
##### Now, lets try to predict two independent distributions ####
# Simple rules:
#   If input is positive:
#     pos_points = input * input
#     neg_points = -0.5 * input
#
#   If input is negative:
#     pos_points = -0.5 * input
#     neg_points = -input * input

print('start')
df['predict_1'] = df['input'].apply(lambda x: x*x if x > 0 else x * -0.5)
df['predict_2'] = df['input'].apply(lambda x: x*x*-1 if x < 0 else -x * 0.5)

print('df.head')
print(df.head())

target = pd.concat([df.pop(x) for x in ['predict_1', 'predict_2']], axis=1)
tfd = tfp.distributions

start
df.head
      input  predict_1  predict_2
0  3.498475  12.239330  -1.749238
1  2.134057   4.554200  -1.067029
2  5.456111  29.769145  -2.728055
3  0.263752   0.069565  -0.131876
4  7.399465  54.752085  -3.699733


In [ ]:
tfd = tfp.distributions
sample_layer = tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[..., :1],
                           scale=1e-3 + tf.math.softplus(0.05 * t[...,1:])))
def get_df_model():
  inputs = tf.keras.layers.Input(shape=[len(df.columns),])
  x = tf.keras.layers.Dense(10, activation='relu')(inputs)
  x = tf.keras.layers.Dense(10, activation='relu')(x)
  outputs1 = tf.keras.layers.Dense(len(target.columns))(x)
  outputs2 = tf.keras.layers.Dense(len(target.columns))(x) # there are 2 outputs, so we want a mean + standard deviation for EACH of the outputs
    
  outputs1 = sample_layer(outputs1)
  outputs2 = sample_layer(outputs2)
  model = tf.keras.Model(inputs, [outputs1, outputs2])

  negloglik = lambda y, rv_y: -rv_y.log_prob(y)

  model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01), loss=negloglik)
  return model

In [ ]:
model = get_df_model()
model.summary()
model.fit(df, target, epochs=10)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 dense_51 (Dense)               (None, 10)           20          ['input_4[0][0]']                
                                                                                                  
 dense_52 (Dense)               (None, 10)           110         ['dense_51[0][0]']               
                                                                                                  
 dense_53 (Dense)               (None, 4)            44          ['dense_52[0][0]']               
                                                                                            